# Read property data

In [1]:
import pandas as pd
import re
import math as math
from itertools import permutations 

In [ ]:
property_data_2022 = pd.read_json('../data/raw/2022_house_data.json')

In [3]:
property_data_2022 = property_data_2022.T
property_data_2022 = property_data_2022.reset_index(drop=True)

# Get postcode from address

In [4]:
def extract_postcode (row):
    if 'VIC' not in row['name']:
        return False
    split_word = 'VIC'
    postcode = row['name'].partition(split_word)[2]
    postcode = postcode.strip()
    if len(postcode) != 4:
        postcode = postcode.partition(split_word)[2]
        postcode = postcode.strip()
    return postcode

In [5]:
# Checking the postcode
property_data_2022.apply (lambda row: extract_postcode(row), axis=1)

0        3000
1        3000
2        3000
3        3000
4        3000
         ... 
15087    3996
15088    3996
15089    3996
15090    3996
15091    3996
Length: 15092, dtype: object

In [6]:
# Extract the two housing data which is not in Victoria
for i in [13560,13561]:
    print(property_data_2022.iloc[i])

name                      3391 Tocumwal-Barooga Road Barooga NSW 3644
cost_text                                            $490.00 per week
property_type                                                   House
property_feature                                      [Available Now]
coordinates                                [-35.9164478, 145.6658604]
rooms                               [[3 Beds], [1 Bath], [4 Parking]]
desc                 class="css-dxogle">* Unverified feature<svg a...
Name: 13560, dtype: object
name                            Lot 3 Buchanans Road Barooga NSW 3644
cost_text                                            $220.00 per week
property_type                                                   House
property_feature                                [Available Now, $880]
coordinates                                [-35.9089932, 145.6994904]
rooms                                        [[2 Beds], [1 Bath], []]
desc                 class="css-dxogle">* Unverified feature<sv

In [7]:
# Assigning the postcode
property_data_2022["postcode"] = property_data_2022.apply (lambda row: extract_postcode(row), axis=1)
property_data_2022 = property_data_2022[property_data_2022.postcode != False]

# Scrapping the SA2 name matches to postcodes

In [8]:
# getting the the table postcodes matches to SA2 region name
# The data scrape from https://www.matthewproctor.com/full_australian_postcodes_vic
import requests
from bs4 import BeautifulSoup
url = "https://www.matthewproctor.com/full_australian_postcodes_vic"
data = requests.get(url).text  
soup = BeautifulSoup(data, 'html.parser') 

In [9]:
table = soup.find("table", {"class": "table table-striped"}) 

In [10]:
df = pd.DataFrame(columns=['postcodes', 'SA2_names'])

# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[1].text.strip()
        SA2_name = columns[19].text.strip()
        df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1975/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1975/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1975/3783446249.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1975/3783446249.py:11: FutureWarning: The frame

In [11]:
sa2_dataframe = df.iloc[1:]

In [12]:
# checking the results we scrapped
sa2_dataframe.head()

,postcodes,SA2_names
1,3000,Melbourne
2,3001,Melbourne
3,3002,East Melbourne
4,3003,West Melbourne
5,3004,Southbank


# Adding the SA2 name to each postcode

In [13]:
def add_sa2(row,sa2_dataframe):
    postcode = row['postcode']
    selected_postcode = sa2_dataframe.loc[sa2_dataframe['postcodes'] == postcode]
    sa2 = [(selected_postcode.iloc[i])["SA2_names"] for i in range(len(selected_postcode))]
    sa2=list(set(sa2))
    return sa2

In [14]:
property_data_2022.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

0                            [Melbourne]
1                            [Melbourne]
2                            [Melbourne]
3                            [Melbourne]
4                            [Melbourne]
                      ...               
15087    [Foster, Wonthaggi - Inverloch]
15088    [Foster, Wonthaggi - Inverloch]
15089    [Foster, Wonthaggi - Inverloch]
15090    [Foster, Wonthaggi - Inverloch]
15091    [Foster, Wonthaggi - Inverloch]
Length: 15090, dtype: object

In [15]:
property_data_2022["sa2_region"] = property_data_2022.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

# Starting the data preprocssing:(2022 data)
1. extract the cost。
2. seperate the number of bedroom, number bathroom and number of parking into an individual new column
3. Adding the SA2 population 2022
4. Adding the SA2 Income 2022
5. Adding the distance to the train station.
6. Adding the distance to school.


In [16]:
# This is the same function with the previous 2017-2021 data-preprocessing function and the coment already add in there.
def extract_cost_number (row):
    cost_string = row['cost_text']
    
   
    cost_string = cost_string.replace('/', ' per ')
    
    cost_list = []
    
   
    for cost in re.findall('\d+\s*\w+\s*\w+',cost_string):
        cost_list.append(cost)
        
    cost_list = [cost.lower() for cost in cost_list]
    
   
    dictOfStrings = {'pw': 'per week', 'weekly': 'week',
                    'pcm':'per month','mth':'month','wek':'week',
                    'pa':'per year','wk':'week','annually':'year',
                    'pwk':'per week','poa':'per year','monthly':'month',
                    'pcw':'per week','weekly':'week','annum':'year','pm':'per month'}
    replace_abb = []
    for cost in cost_list:
        for word, replacement in dictOfStrings.items():
            cost = re.sub(word, replacement, cost)
        replace_abb.append(cost)     
    if len(replace_abb) > 0:
        cost_type = replace_abb[0]
        
        dictOfAlpha = {'h':'hour','d':'day','w':'week','m':'month','y':'year'}
        processed_test = ''
        for w in cost_type.split():
            if (len(w) == 1) and (w in dictOfAlpha):
                processed_test += dictOfAlpha[w]
            else:
                processed_test += w
            processed_test += " "
        rent_type = ''.join([i for i in processed_test if not i.isdigit()])
        cost = ''.join([i for i in processed_test if i.isdigit()])
        if 'hour' in rent_type:
            rent_per_week = float(cost)*24*7
        elif 'day' in rent_type:
            rent_per_week = float(cost)*7    
        elif 'month' in rent_type:
            rent_per_week = (float(cost)/30)*7   
        elif 'year' in rent_type:
            rent_per_week = (float(cost)/365)*7  
        elif 'fortnightly' in rent_type:
            rent_per_week = (float(cost)/2)  
        else:
            rent_per_week = float(cost)
        return rent_per_week
    else:
        return "No Available"  

In [17]:
#add the weekly rent into a new column
property_data_2022['week_rent'] = property_data_2022.apply (lambda row: extract_cost_number(row), axis=1)

In [18]:
# use the first SA2 area as the area which the house belows to 
SA2_centroids = pd.read_csv('../data/raw/mb_2016_centroids.csv')
all_value = []
SA2_vic = SA2_centroids[SA2_centroids['STE_NAME16']=='Victoria']
SA2_vic = SA2_vic[['SA2_MAIN16','SA2_NAME16','xcoord','ycoord']]
SA2_vic.head()
SA2_vic_xcoord = SA2_vic.groupby('SA2_NAME16')["xcoord"].mean()
SA2_vic_ycoord = SA2_vic.groupby('SA2_NAME16')["ycoord"].mean()
SA2_vic_xy = pd.merge(SA2_vic_xcoord,SA2_vic_ycoord,left_on = 'SA2_NAME16',right_on = 'SA2_NAME16')
#groupy the SA2_NAME16 and take the average of xcoord and ycoord
SA2_vic_xy=dict(SA2_vic_xy.transpose())
all_co = []
for value in property_data_2022['sa2_region']:
    ind_co=[] 
    for region in value:
        if region in SA2_vic_xy.keys():
            ind_co.append(SA2_vic_xy[region])
    all_co.append(ind_co)
property_data_2022['sa2_coor'] = all_co
property_data_2022.head()

,name,cost_text,property_type,property_feature,coordinates,rooms,desc,postcode,sa2_region,week_rent,sa2_coor
0,101/25-33 Wills Street Melbourne VIC 3000,$400,Apartment / Unit / Flat,"[Friday, 7th October 2022, $1738]","[-37.8107757, 144.9569269]","[[1 Bed], [1 Bath], []]",Mary Khudhur,3000,[Melbourne],400.0,"[[144.9627648781132, -37.813376920301884]]"
1,2905B/11 Rose Lane Melbourne VIC 3000,$575 per week,Apartment / Unit / Flat,"[Tuesday, 20th September 2022, $2499]","[-37.815446, 144.9541165]","[[2 Beds], [1 Bath], [1 Parking]]",Mikaele Horsnell,3000,[Melbourne],575.0,"[[144.9627648781132, -37.813376920301884]]"
2,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,Apartment / Unit / Flat,"[Available Now, $10800]","[-37.8318086, 144.9710272]","[[3 Beds], [2 Baths], [2 Parking]]",Darren McMullin,3000,[Melbourne],1800.0,"[[144.9627648781132, -37.813376920301884]]"
3,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,Apartment / Unit / Flat,"[Tuesday, 18th October 2022, $2085]","[-37.816228, 144.9532465]","[[1 Bed], [1 Bath], [1 Parking]]",South Melbourne Leasing,3000,[Melbourne],480.0,"[[144.9627648781132, -37.813376920301884]]"
4,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,Apartment / Unit / Flat,"[Friday, 9th September 2022, $1738]","[-37.8121026, 144.9581291]","[[1 Bed], [1 Bath], []]",Shaun Broadbent,3000,[Melbourne],400.0,"[[144.9627648781132, -37.813376920301884]]"


In [19]:
#function to calculate the distance 
def getDistance(latA, lonA, latB, lonB):
    ra = 6378140  
    rb = 6356755  
    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)

    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB)) ** 2 / math.cos(x / 2) ** 2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB)) ** 2 / math.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance = round(distance / 1000, 4)
    return distance

In [20]:
# Find the closest SA2 
def close_sa2(row): 
    all_distance = [getDistance(row['coordinates'][0],row['coordinates'][1],x[1],x[0]) for x in row.sa2_coor]
    min_distance = min(all_distance,default = 0)
    return row['sa2_region'][all_distance.index(min_distance)]   

In [21]:
property_data_2022['closest_sa2'] = property_data_2022.apply(lambda row:close_sa2(row),axis=1)

In [22]:
property_data_2022.head()

,name,cost_text,property_type,property_feature,coordinates,rooms,desc,postcode,sa2_region,week_rent,sa2_coor,closest_sa2
0,101/25-33 Wills Street Melbourne VIC 3000,$400,Apartment / Unit / Flat,"[Friday, 7th October 2022, $1738]","[-37.8107757, 144.9569269]","[[1 Bed], [1 Bath], []]",Mary Khudhur,3000,[Melbourne],400.0,"[[144.9627648781132, -37.813376920301884]]",Melbourne
1,2905B/11 Rose Lane Melbourne VIC 3000,$575 per week,Apartment / Unit / Flat,"[Tuesday, 20th September 2022, $2499]","[-37.815446, 144.9541165]","[[2 Beds], [1 Bath], [1 Parking]]",Mikaele Horsnell,3000,[Melbourne],575.0,"[[144.9627648781132, -37.813376920301884]]",Melbourne
2,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,Apartment / Unit / Flat,"[Available Now, $10800]","[-37.8318086, 144.9710272]","[[3 Beds], [2 Baths], [2 Parking]]",Darren McMullin,3000,[Melbourne],1800.0,"[[144.9627648781132, -37.813376920301884]]",Melbourne
3,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,Apartment / Unit / Flat,"[Tuesday, 18th October 2022, $2085]","[-37.816228, 144.9532465]","[[1 Bed], [1 Bath], [1 Parking]]",South Melbourne Leasing,3000,[Melbourne],480.0,"[[144.9627648781132, -37.813376920301884]]",Melbourne
4,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,Apartment / Unit / Flat,"[Friday, 9th September 2022, $1738]","[-37.8121026, 144.9581291]","[[1 Bed], [1 Bath], []]",Shaun Broadbent,3000,[Melbourne],400.0,"[[144.9627648781132, -37.813376920301884]]",Melbourne


# Merge the income and population into the dataset using SA2

In [23]:
#merge the population
all_population = pd.read_csv('../data/raw/population_final_predict.csv')
all_population = all_population.drop(columns=['Unnamed: 0'])
melbourne_population = all_population[all_population['SA2 name'].str.contains('Melbourne CBD')].sum(axis=0)
melbourne_population['SA2 name'] = 'Melbourne'
all_population = all_population.append(melbourne_population,ignore_index=True)

all_population_2022 = all_population[['2022 Population','SA2 name']]
all_population_2022.head()

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1975/3523860934.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_population = all_population.append(melbourne_population,ignore_index=True)


,2022 Population,SA2 name
0,16623.337207,Alfredton
1,12366.053200,Ballarat
2,7749.168968,Buninyong
3,10722.789138,Delacombe
4,4176.433099,Smythes Creek


In [24]:
#merge the income By SA2
all_income = pd.read_csv('../data/raw/Income_Prediction.csv')

# Manually filling the income of Melbourne 
# https://www.averagesalarysurvey.com/melbourne-australia
data = {'SA2 code': [206041122],
        'SA2 name': ['Melbourne'],
        '2010': [64792],
        '2011':[69316],
        '2012':[71532],
        '2013':[72800],
        '2014':[74827],
        '2015':[76189],
        '2017Pred':[87360],
        '2018Pred':[91230],
        '2019Pred':[99820],
        '2020Pred':[116530],
        '2021Pred':[120000],
        '2022Pred':[126197],
        '2023Pred':[129324],
        '2024Pred':[134234],
        '2026Pred':[136249],
        '2027Pred':[140124]}  
# Create DataFrame  
df = pd.DataFrame(data)  
all_income = all_income.append(df)

all_income_2022 = all_income[['2022Pred','SA2 name']]
all_income_2022.head()

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1975/1668022609.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_income = all_income.append(df)


,2022Pred,SA2 name
0,68727.942857,Alfredton
1,91276.600000,Ballarat
2,69361.038095,Buninyong
3,61397.285714,Delacombe
4,63208.438095,Smythes Creek


In [25]:
property_data_2022 = pd.merge(property_data_2022,all_population_2022,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
property_data_2022 = pd.merge(property_data_2022,all_income_2022,how='inner',left_on='closest_sa2',right_on='SA2 name')

In [26]:
def data_preprocessing_22(data_set):
    #extract the price per week out
    bed_num=[]
    bath_num=[]
    park_num=[]
    #seperate the bedroom, bathroom and parking area
    for value in data_set['rooms']:
        bed_num.append(re.findall(r'(\d+) Bed', str(value))) #找到bed前面的数字
        bath_num.append(re.findall(r'(\d+) Bath', str(value)))
        park_num.append(re.findall(r'(\d+) Parking', str(value)))
    data_set['bedroom_num'] = bed_num
    data_set['bathroom_num'] = bath_num
    data_set['parking_num'] = park_num
    data_set = data_set.drop(['cost_text'],axis=1)
    data_set = data_set.drop(['rooms'],axis=1)
    data_set = data_set.drop(['sa2_region'],axis=1)
    data_set = data_set.drop(['SA2 name_x'],axis=1)
    data_set = data_set.drop(['SA2 name_y'],axis=1)
    data_set = data_set.drop(['property_feature'],axis=1)
    data_set = data_set.drop(['desc'],axis=1)
    return data_set
property_data_2022=data_preprocessing_22(property_data_2022) 

In [27]:
#change form of information
def extract_bed(row):
    if row['bedroom_num'] == []:
        return None
    return row['bedroom_num'][0]
def extract_bath(row):
    if row['bathroom_num'] == []:
        return None
    return row['bathroom_num'][0]
def extract_car(row):
    if row['parking_num'] == []:
        return None
    return row['parking_num'][0]
property_data_2022['bedroom_num'] = property_data_2022.apply(lambda row:extract_bed(row),axis=1)
property_data_2022['bathroom_num'] = property_data_2022.apply(lambda row:extract_bath(row),axis=1)
property_data_2022['parking_num'] = property_data_2022.apply(lambda row:extract_car(row),axis=1)
property_data_2022.rename({'2022 Population':'Population'},axis=1,inplace=True)
property_data_2022.rename({'2022Pred':'Income'},axis=1,inplace=True)
property_data_2022.rename({'bedroom_num':'bed'},axis=1,inplace=True)
property_data_2022.rename({'bathroom_num':'bath'},axis=1,inplace=True)
property_data_2022.rename({'parking_num':'car'},axis=1,inplace=True)
property_data_2022['years'] = '2022'

In [28]:
property_data_2022 = property_data_2022[['name','property_type','years','coordinates','postcode','sa2_coor','closest_sa2','bed','bath','car','Income','Population','week_rent']]

In [29]:
property_data_2022.head()

,name,property_type,years,coordinates,postcode,sa2_coor,closest_sa2,bed,bath,car,Income,Population,week_rent
0,101/25-33 Wills Street Melbourne VIC 3000,Apartment / Unit / Flat,2022,"[-37.8107757, 144.9569269]",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,1,1,None,126197.0,47618.4,400.0
1,2905B/11 Rose Lane Melbourne VIC 3000,Apartment / Unit / Flat,2022,"[-37.815446, 144.9541165]",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,2,1,1,126197.0,47618.4,575.0
2,1901/368 St Kilda Road Melbourne VIC 3000,Apartment / Unit / Flat,2022,"[-37.8318086, 144.9710272]",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,3,2,2,126197.0,47618.4,1800.0
3,1211/200 Spencer Street Melbourne VIC 3000,Apartment / Unit / Flat,2022,"[-37.816228, 144.9532465]",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,1,1,1,126197.0,47618.4,480.0
4,1008/380 Little Lonsdale Street Melbourne VIC ...,Apartment / Unit / Flat,2022,"[-37.8121026, 144.9581291]",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,1,1,None,126197.0,47618.4,400.0


In [30]:
property_data_2022.to_csv('../data/raw/2022_house_data.csv')